In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/TalkLikeMom/src/Classifier

/content/drive/MyDrive/PHD/TalkLikeMom/src/Classifier


In [ ]:
# !pip install -q transformers pythainlp datasets evaluate sentencepiece
# # !pip install -q accelerate -U
# !pip install -q transformers[torch]
# !pip install -q shap nlp

In [ ]:
!pip install spacy-alignments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('.')

In [ ]:
from data_loader import get_task1_conver, get_task2_conver
from utils import dump_jsonl, load_jsonl


import pickle

def load_shap_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

def save_shap_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)

In [ ]:
import pandas as pd

In [ ]:
# from transformers import AutoTokenizer
# model_name = "airesearch/wangchanberta-base-att-spm-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

## Model Performance

In [ ]:
import json
import numpy as np

def load_json(filename):
  with open(filename) as fin:
    content = json.load(fin)

  return content

for m in ["wc", "py", "xl"]:
  for task in ["clse", "auth"]:
    for setting in ["task1", "task2", "task3"]:
      values = []
      k = "eval_f1"
      taskName = f"{m}_{setting}_{task}"
      for idx in range(5):
        folderName = f"Regressors/{taskName}_usr{idx}/eval.json"
        evalResults = load_json(folderName)

        # Remove cases where the model loss is out-of-control
        if abs(evalResults["val"][k] - evalResults["test"][k]) > 0.155:
          continue
        values.append(evalResults["test"][k])

      avg = np.mean(values)
      std = np.std(values)
      print(taskName, f"{avg:.3f}±{std:.3f}")

wc_task1_clse 0.657±0.072
wc_task2_clse 0.490±0.048
wc_task3_clse 0.639±0.058
wc_task1_auth 0.313±0.056
wc_task2_auth 0.748±0.051
wc_task3_auth 0.761±0.029
py_task1_clse 0.666±0.016
py_task2_clse 0.496±0.009
py_task3_clse 0.657±0.013
py_task1_auth 0.431±0.044
py_task2_auth 0.750±0.017
py_task3_auth 0.712±0.000
xl_task1_clse 0.604±0.053
xl_task2_clse 0.420±0.086
xl_task3_clse 0.498±0.043
xl_task1_auth 0.200±0.064
xl_task2_auth 0.675±0.089
xl_task3_auth 0.432±0.071


## Load Lexicons

In [ ]:
from pythainlp.tokenize import word_tokenize
import numpy as np

In [ ]:
import json
with open("../words.json", encoding="utf-8") as fin:
    raw = json.load(fin)
    thaidict_royal = set()
    for k in raw:
        thaidict_royal.update(raw[k])

In [ ]:
lexicons_arr = load_jsonl("../lexicons.jsonl")

Loaded 25603 records from ../lexicons.jsonl


In [ ]:
from collections import defaultdict
tags = set()
lexicons = {}
lexicons_keys = defaultdict(list)

for key, values  in lexicons_arr:
    if len(key) <= 1:
        continue

    key = key.lower()
    if key.endswith("rep"):
        key = key.replace("rep", "")

    w = word_tokenize(key)

    lexicons_keys[w[0]].append(key)

    tag = [t for t in values["tags"] if not t.startswith("cat:")]
    lexicons[key] = tag
    tags.update(tag)

In [ ]:
# cc = 0
# for k in lexicons:
#     for t in lexicons[k]:
#         if t=="transliterated":
#             cc += 1
# cc

In [ ]:
metric_names = {
    "Reference" : {
#         "all": "All words",
        "pertoken": "Average per token"
    },
#     "Linguistic Complexity" : {
#         "nunique": "Vocabulary size",
#         "nthai": "Thai words",
#         "nnotthai": "Non-Thai words",
#         "nlongword": "Long words",
#         "ndict": "Dictionary words",
#         "transliterated": "Transliteration",
#     },
    "Pronoun": {
        "pronoun": "All pronoun",
        "pronoun_1st": ">> 1st person pronoun",
        "pronoun_2nd": ">> 2nd person pronoun",
        "pronoun_3rd": ">> 3rd person pronoun",
        "pronoun_singular": ">> Singular pronoun",
        "pronoun_plural": ">> Plural pronoun",
        "pronoun_misspelling": ">> Pronoun in non-standard spelling",
    },

    "Sentence-ending Particles": {
        "particles": "All particles",
        "particles_SARP": ">> Socially-related particles",
        "particles_notSARP": ">> Non-socially-related particles",
        "particles_misspelling": ">> Particle in non-standard spelling",
    },

#     "Sentiment-related": {
#         "sentiment": "Sentiment words",
#         "sentiment_positive": ">> Positive words",
#         "sentiment_negative": ">> Negative words",
#     },

    "Spelling Variation": {
        "misspelling": "All spelling variation",
        "misspelling_common": ">> Common misspelt words",
        "misspelling_intention": ">> Morphophonemic variation",
        "misspelling_shorten": ">> Simplified variation",
        "nrepeat": ">> Repeated characters",
#         "nemoji": ">> Emoji",
#         "abbr": "Abbreviation",
#         "slang": "Slang",
#         "swear": "Swear words"
    }
}

## Load SHAP Values

In [ ]:
from collections import defaultdict
from itertools import groupby
from data_loader import preprocess

import spacy_alignments as tokenizations


def get_shap_lexicons(df, raw_shap_values):
    shap_lexicons = {"overall": []}
    label_values = set(df["label"])



    # _tmp = raw_shap_values[:, :]
    shap_data = raw_shap_values.data
    shap_values = raw_shap_values.values

    for _, label in enumerate(label_values):
        feats = []
        for idx, (l, t) in enumerate(zip(df["label"], df["text"])):
            if l!=label:
                continue

            words = word_tokenize(preprocess(t))
            words = [w.strip() for w in words if len(w.strip())>0]

            shap_tokens = [(w.strip(), v) for w,v in zip(shap_data[idx], shap_values[idx]) if len(w.strip())>0]
            tokens = [t for t, _ in shap_tokens]
            values = [v for _, v in shap_tokens]

            w2t, t2w = tokenizations.get_alignments(words, tokens)
            # shap_tokens = map_token_2_words(words, shap_tokens, debug=False)
            newwords = []
            newvalues = []
            for widx, word in enumerate(w2t):
              w = ""
              v = 0
              for tidx in word:
                w += tokens[tidx]
                v += values[tidx]
              newwords.append(w)
              newvalues.append(v)

            feats.append((newwords, newvalues))
        shap_lexicons[label] = feats
        shap_lexicons["overall"] += feats

    return shap_lexicons

def is_target_lexicons(token):
  token = token.lower()
  if token=="rep":
      return True

  if token in lexicons_keys:
      for l in lexicons_keys[token]:
          if not token.startswith(l):
              continue

          return True
  return False

def get_shap_by_lexicons(shap_path, fn_abs=False):
    shap_values = load_shap_values(shap_path)

    shap_lexicons = get_shap_lexicons(shap_values["inputs"], shap_values["values"])
    lexicons = {}
    for label in shap_lexicons:

      shapley_values = defaultdict(list)
      for words, shap in shap_lexicons[label]:
        for w, s in zip(words, shap):
          shapley_values[w].append(s)


      feature_names = []
      avg_shape_values = []
      for w in shapley_values:
        if not is_target_lexicons(w):
          continue

        feature_names.append(w)

        if fn_abs:
            s = np.mean(np.abs(shapley_values[w]))
        else:
            s = np.mean((shapley_values[w]))

        avg_shape_values.append(s)

      sorted_shap_values = sorted(zip(avg_shape_values, feature_names), key=lambda pair: -pair[0])
      lexicons[label] = {x.lower():v for v, x in sorted_shap_values}

    return lexicons




In [ ]:
lexicons["rep"] = ["misspelling"]

def get_lexicons_by_feat(shap_lexicons):
  shap_lexicons_by_feat = {}
  for label in shap_lexicons:
    feats = ["pronoun", "particles", "misspelling"]
    shap_lexicons_by_feat[label] = {}
    for f in feats:
      shap_lexicons_by_feat[label][f] = {}
      for w in shap_lexicons[label]:
        tags = lexicons[w]

        if f in lexicons[w]:
          shap_lexicons_by_feat[label][f][w] = shap_lexicons[label][w]
  return shap_lexicons_by_feat

In [ ]:
!mv ShapleyValues/xl_task2_clas_usr.pkl ShapleyValues/xl_task2_clse_usr.pkl

In [ ]:
fn_abs = True
suffix = "abs" if fn_abs else ''

for m in ["wc", "py", "xl"]:
  for task in ["clse", "auth"]:
    for setting in ["task1", "task2", "task3"]:
      values = []
      k = "eval_f1"
      taskName = f"{m}_{setting}_{task}"
      print(f"runing... {taskName}")
      shap_lexicons = get_shap_by_lexicons(f"./ShapleyValues/{taskName}_usr.pkl", fn_abs)
      shap_lexicons_by_feat = get_lexicons_by_feat(shap_lexicons)

      dump_jsonl(f"Lexicons/{taskName}_{suffix}.jsonl", [shap_lexicons, shap_lexicons_by_feat])

runing... wc_task1_clse
Wrote 2 records to Lexicons/wc_task1_clse_abs.jsonl
runing... wc_task2_clse
Wrote 2 records to Lexicons/wc_task2_clse_abs.jsonl
runing... wc_task3_clse
Wrote 2 records to Lexicons/wc_task3_clse_abs.jsonl
runing... wc_task1_auth
Wrote 2 records to Lexicons/wc_task1_auth_abs.jsonl
runing... wc_task2_auth
Wrote 2 records to Lexicons/wc_task2_auth_abs.jsonl
runing... wc_task3_auth
Wrote 2 records to Lexicons/wc_task3_auth_abs.jsonl
runing... py_task1_clse
Wrote 2 records to Lexicons/py_task1_clse_abs.jsonl
runing... py_task2_clse
Wrote 2 records to Lexicons/py_task2_clse_abs.jsonl
runing... py_task3_clse
Wrote 2 records to Lexicons/py_task3_clse_abs.jsonl
runing... py_task1_auth
Wrote 2 records to Lexicons/py_task1_auth_abs.jsonl
runing... py_task2_auth
Wrote 2 records to Lexicons/py_task2_auth_abs.jsonl
runing... py_task3_auth
Wrote 2 records to Lexicons/py_task3_auth_abs.jsonl
runing... xl_task1_clse
Wrote 2 records to Lexicons/xl_task1_clse_abs.jsonl
runing... xl

## Print outputs

In [ ]:
from pythainlp.util import countthai
import re
# import emoji

def get_lexicon_feats(token, ref_text):
    feats = ["all"]

    if token=="rep":
        feats.append("nrepeat")

    if token in lexicons_keys:
        for l in lexicons_keys[token]:
            if not ref_text.startswith(l):
                continue

            feats.extend(lexicons[l])

    if token in thaidict_royal:
        feats.append("ndict")

    if len(token) > 7:
        feats.append("nlongword")

    if countthai(token) < 50:
        nt = re.sub(r'\W+', '', token)
        if token not in ["usr", "sys", "rep"] and len(nt) > 0 and not nt.isnumeric():
            feats.append("nnotthai")
    else:
        feats.append("nthai")

    if "particles" in feats and "particles_SARP" not in feats:
        feats.append("particles_notSARP")

    # if emoji.emoji_count(token) > 0:
    #     feats.append("nemoji")

    return feats


def get_shap_feats(shap_lexicons):
    output = {}
    for label in shap_lexicons:
        all_shap_feats = []
        for tokens, values in shap_lexicons[label]:
            # shap_feats is per conversation
            shap_feats = defaultdict(list)
            for tidx, (t, v) in enumerate(zip(tokens, values)):
                feats = get_lexicon_feats(t, "".join(tokens[tidx:]))

                for f in feats:
                    shap_feats[f].append(v)

#             shap_feats["pertoken"] = sum(values)/len(values)
            shap_feats["pertoken"] = values
            all_shap_feats.append(shap_feats)

        mean_shap_feats = {}
        for g in metric_names:
            for m in metric_names[g]:
                values_each_token = []
                values_each_conv = []

                for feats in all_shap_feats:
                    if m in feats:
                        values_each_token += feats[m]

                        absum = np.sum(np.abs(np.array(feats[m])))
                        values_each_conv += [absum]
                        #values.append(feats[m])

                if len(values_each_token)==0:
                    mean_shap_feats[m] = (0, 0, 0, 0)
                    continue

                values = np.array(values_each_token)
                pertoken = np.mean(np.abs(np.array(values)))


                perconv = np.mean(np.array(values_each_conv))

                mean_shap_feats[m] = (pertoken, len(values_each_token), perconv, len(values_each_conv))
        output[label] = mean_shap_feats

    return output

def run_lexicons(shap_path):
    shap_values = load_shap_values(shap_path)

    shap_lexicons = get_shap_lexicons(shap_values["inputs"], shap_values["values"])
    shap_feats = get_shap_feats(shap_lexicons)

    return shap_feats



In [ ]:
def outputs_to_dict(outputs):
    coefs = {}
    all_coefs = {}

    for s in outputs:
        for feat in outputs[s]:
            if feat not in coefs:
                coefs[feat] = (0, 0)

            if feat not in all_coefs:
                all_coefs[feat] = (0, 0)

            # pertoken coefs
            val1, n1, _, _ = outputs[s][feat]
            val2, n2 = coefs[feat]

            if n1+n2 == 0:
                coefs[feat] = (0, 0)
            else:
                val = (val1*n1 + val2*n2)*1.0/(n1+n2)
                n = n1 + n2
                coefs[feat] = (val, n)

            # perconv coefs
            _, _, val1, n1 = outputs[s][feat]
            val2, n2 = all_coefs[feat]

            if n1+n2 == 0:
                all_coefs[feat] = (0, 0)
            else:
                val = (val1*n1 + val2*n2)*1.0/(n1+n2)
                n = n1 + n2
                all_coefs[feat] = (val, n)

    return coefs, all_coefs



In [ ]:
def print_tables(shap_feats):
    printed_text = ""
    printed_text += "\subsection{Closeness}"+"\n"
    outputs = [
        outputs_to_dict(shap_feats[1]),
        outputs_to_dict(shap_feats[3]),
        outputs_to_dict(shap_feats[5])
    ]

    printed_text += "\\begin{longtable}[h]{"+"\n"
    printed_text += "    p{\dimexpr 0.40\\linewidth-2\\tabcolsep}|c|c|c|c|c|c|"+"\n"
    printed_text += "}"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += "    Lexical Features & " + "\n"
    printed_text += "    \\multicolumn{2}{|c|}{Setting 1} & " + "\n"
    printed_text += "    \\multicolumn{2}{|c|}{Setting 2} & " + "\n"
    printed_text += "    \\multicolumn{2}{|c|}{Setting 3} \\\\" + "\n"
    printed_text += "    \\cline{2-7}" + "\n"
    printed_text += "    & " + "\n"
    printed_text += "    Per \\newline token & Total & " + "\n"
    printed_text += "    Per \\newline token & Total & " + "\n"
    printed_text += "    Per \\newline token & Total \\\\" + "\n"

    printed_text += "    \hline"+"\n"
    #     printed_text += "    \endfirsthead"+"\n"
    #     printed_text += ""+"\n"
    printed_text += "    \endhead"+"\n"
    printed_text += ""+"\n"

    # for sec, results in zip(sections, outputs):
    for g in metric_names:
        if g in ["Conversational Statistics"]:
            continue

        printed_text += "    \multicolumn{7}{l}{\\textit{"+g+"}} \\\\"+"\n"
        printed_text += "    \hline"+"\n"

        for m in metric_names[g]:
            s = f"        {metric_names[g][m]} "
            for out, all_out in outputs:
                if m not in out:
                    s += f"& - "
                else:
                    val, n = out[m]
                    all_val, _ = all_out[m]
                    ref, _ = out["pertoken"]

                    if (val - ref) > 0.1*ref:
                        s += "& \cellcolor{gray!25} "+f"{val*100:.2f}"+" & \cellcolor{gray!25} "+f"{all_val*100:.2f}"
                    else:
                        s += f"& {val*100:.2f} & {all_val*100:.2f}"

            s += "\\\\"
            # print(s)
            printed_text += s+"\n"


        printed_text += "        & & & & & &\\\\"+"\n"
        printed_text += "    \hline"+"\n"
        printed_text += ""+"\n"

    printed_text += "\label{closeness_wangchanberta_shapley_value}"+"\n"
    printed_text += "\end{longtable}"+"\n"
    printed_text += "\clearpage"+"\n"

    printed_text += "\subsection{Respect}"+"\n"

    outputs = [
        outputs_to_dict(shap_feats[2]),
        outputs_to_dict(shap_feats[4]),
        outputs_to_dict(shap_feats[6])
    ]

    printed_text += "\\begin{longtable}[h]{"+"\n"
    printed_text += "    p{\dimexpr 0.40\\linewidth-2\\tabcolsep}|c|c|c|c|c|c|"+"\n"
    printed_text += "}"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += "    Lexical Features & " + "\n"
    printed_text += "    \\multicolumn{2}{|c|}{Setting 1} & " + "\n"
    printed_text += "    \\multicolumn{2}{|c|}{Setting 2} & " + "\n"
    printed_text += "    \\multicolumn{2}{|c|}{Setting 3} \\\\" + "\n"
    printed_text += "    \\cline{2-7}" + "\n"
    printed_text += "    & " + "\n"
    printed_text += "    Per \\newline token & Total & " + "\n"
    printed_text += "    Per \\newline token & Total & " + "\n"
    printed_text += "    Per \\newline token & Total \\\\" + "\n"

    printed_text += "    \hline"+"\n"
    #     printed_text += "    \endfirsthead"+"\n"
    #     printed_text += ""+"\n"
    printed_text += "    \endhead"+"\n"
    printed_text += ""+"\n"

    # for sec, results in zip(sections, outputs):
    for g in metric_names:
        if g in ["Conversational Statistics"]:
            continue

        printed_text += "    \multicolumn{7}{l}{\\textit{"+g+"}} \\\\"+"\n"
        printed_text += "    \hline"+"\n"

        for m in metric_names[g]:
            s = f"        {metric_names[g][m]} "
            for out, all_out in outputs:
                if m not in out:
                    s += f"& - "
                else:
                    val, n = out[m]
                    all_val, _ = all_out[m]
                    ref, _ = out["pertoken"]

                    if (val - ref) > 0.1*ref:
                        s += "& \cellcolor{gray!25} "+f"{val*100:.2f}"+" & \cellcolor{gray!25} "+f"{all_val*100:.2f}"
                    else:
                        s += f"& {val*100:.2f} & {all_val*100:.2f}"

            s += "\\\\"
            # print(s)
            printed_text += s+"\n"


        printed_text += "        & & & & & &\\\\"+"\n"
        printed_text += "    \hline"+"\n"
        printed_text += ""+"\n"
    printed_text += "\label{respect_wangchanberta_shapley_value}"+"\n"
    printed_text += "\end{longtable}"+"\n"

    return printed_text

In [ ]:
# shap_feats1 = run_lexicons(f"./ShapleyValuesV3/task1_clse_regressor.pkl")
# shap_feats2 = run_lexicons(f"./ShapleyValuesV3/task1_auth_regressor.pkl")
# shap_feats3 = run_lexicons(f"./ShapleyValuesV3/task2_clse_regressor.pkl")
# shap_feats4 = run_lexicons(f"./ShapleyValuesV3/task2_auth_regressor.pkl")
# shap_feats5 = run_lexicons(f"./ShapleyValuesV3/task3_clse_regressor.pkl")
# shap_feats6 = run_lexicons(f"./ShapleyValuesV3/task3_auth_regressor.pkl")

In [ ]:
ls ShapleyValues

py_task1_auth_usr.pkl  py_task3_clse_usr.pkl  wc_task3_auth_usr.pkl  xl_task2_clse_usr.pkl
py_task1_clse_usr.pkl  wc_task1_auth_usr.pkl  wc_task3_clse_usr.pkl  xl_task3_auth_usr.pkl
py_task2_auth_usr.pkl  wc_task1_clse_usr.pkl  xl_task1_auth_usr.pkl  xl_task3_clse_usr.pkl
py_task2_clse_usr.pkl  wc_task2_auth_usr.pkl  xl_task1_clse_usr.pkl
py_task3_auth_usr.pkl  wc_task2_clse_usr.pkl  xl_task2_auth_usr.pkl


In [ ]:
fn_abs = True
suffix = "abs" if fn_abs else ''

printed_texts = []
for m in ["wc", "py", "xl"]:
  shap_feats = {}
  idx = 1
  for setting in ["task1", "task2", "task3"]:
    for task in ["clse", "auth"]:
      taskName = f"{m}_{setting}_{task}"
      print(f"runing... {taskName}")
      shap_feats[idx] = run_lexicons(f"./ShapleyValues/{taskName}_usr.pkl")
      idx += 1

  t = print_tables(shap_feats)
  printed_texts.append(t)
  print("Loaded", m)

runing... wc_task1_clse
runing... wc_task1_auth
runing... wc_task2_clse
runing... wc_task2_auth
runing... wc_task3_clse
runing... wc_task3_auth
Loaded wc
runing... py_task1_clse
runing... py_task1_auth
runing... py_task2_clse
runing... py_task2_auth
runing... py_task3_clse
runing... py_task3_auth
Loaded py
runing... xl_task1_clse
runing... xl_task1_auth
runing... xl_task2_clse
runing... xl_task2_auth
runing... xl_task3_clse
runing... xl_task3_auth
Loaded xl


In [ ]:
print("wc", printed_texts[0])

wc \subsection{Closeness}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|c|c|c|c|c|c|
}
    \hline
    Lexical Features & 
    \multicolumn{2}{|c|}{Setting 1} & 
    \multicolumn{2}{|c|}{Setting 2} & 
    \multicolumn{2}{|c|}{Setting 3} \\
    \cline{2-7}
    & 
    Per \newline token & Total & 
    Per \newline token & Total & 
    Per \newline token & Total \\
    \hline
    \endhead

    \multicolumn{7}{l}{\textit{Reference}} \\
    \hline
        Average per token & 1.34 & 156.00& 2.91 & 105.40& 1.17 & 135.70\\
        & & & & & &\\
    \hline

    \multicolumn{7}{l}{\textit{Pronoun}} \\
    \hline
        All pronoun & \cellcolor{gray!25} 1.51 & \cellcolor{gray!25} 5.42& \cellcolor{gray!25} 3.68 & \cellcolor{gray!25} 7.72& \cellcolor{gray!25} 1.92 & \cellcolor{gray!25} 6.78\\
        >> 1st person pronoun & \cellcolor{gray!25} 1.61 & \cellcolor{gray!25} 3.69& \cellcolor{gray!25} 4.51 & \cellcolor{gray!25} 6.76& \cellcolor{gray!25} 1.67 & \cellcolor{gray!25} 3.77\

In [ ]:
print("py", printed_texts[1])

py \subsection{Closeness}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|c|c|c|c|c|c|
}
    \hline
    Lexical Features & 
    \multicolumn{2}{|c|}{Setting 1} & 
    \multicolumn{2}{|c|}{Setting 2} & 
    \multicolumn{2}{|c|}{Setting 3} \\
    \cline{2-7}
    & 
    Per \newline token & Total & 
    Per \newline token & Total & 
    Per \newline token & Total \\
    \hline
    \endhead

    \multicolumn{7}{l}{\textit{Reference}} \\
    \hline
        Average per token & 1.08 & 125.36& 4.07 & 147.01& 0.85 & 97.91\\
        & & & & & &\\
    \hline

    \multicolumn{7}{l}{\textit{Pronoun}} \\
    \hline
        All pronoun & 1.13 & 4.05& \cellcolor{gray!25} 4.52 & \cellcolor{gray!25} 9.47& \cellcolor{gray!25} 1.60 & \cellcolor{gray!25} 5.65\\
        >> 1st person pronoun & \cellcolor{gray!25} 1.25 & \cellcolor{gray!25} 2.85& \cellcolor{gray!25} 5.15 & \cellcolor{gray!25} 7.73& \cellcolor{gray!25} 1.14 & \cellcolor{gray!25} 2.56\\
        >> 2nd person pronoun & \cellco

In [ ]:
print("xl", printed_texts[2])

xl \subsection{Closeness}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|c|c|c|c|c|c|
}
    \hline
    Lexical Features & 
    \multicolumn{2}{|c|}{Setting 1} & 
    \multicolumn{2}{|c|}{Setting 2} & 
    \multicolumn{2}{|c|}{Setting 3} \\
    \cline{2-7}
    & 
    Per \newline token & Total & 
    Per \newline token & Total & 
    Per \newline token & Total \\
    \hline
    \endhead

    \multicolumn{7}{l}{\textit{Reference}} \\
    \hline
        Average per token & 1.07 & 123.94& 1.80 & 65.14& 0.58 & 67.67\\
        & & & & & &\\
    \hline

    \multicolumn{7}{l}{\textit{Pronoun}} \\
    \hline
        All pronoun & 0.80 & 3.64& 1.29 & 3.46& 0.25 & 1.13\\
        >> 1st person pronoun & 0.78 & 2.14& 1.21 & 2.20& 0.23 & 0.63\\
        >> 2nd person pronoun & 0.87 & 2.78& 1.23 & 2.67& 0.28 & 0.88\\
        >> 3rd person pronoun & 0.49 & 1.02& 1.02 & 1.92& 0.20 & 0.41\\
        >> Singular pronoun & 0.80 & 3.63& 1.27 & 3.37& 0.25 & 1.13\\
        >> Plural pronoun 